<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/GA_random_0516.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import

In [ ]:
import csv
import pandas as pd
import numpy as np
import random
import time
import bisect
from itertools import chain

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


## 讀 prime table

In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


## 讀goldbach table

In [ ]:
goldBachTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_1000.csv")
goldBachNumber = list(goldBachTable["Number"])
goldBachPartitions = list(goldBachTable["Factors"])

In [ ]:
def IsPrime(num):
    """檢查一個數字是否為質數"""
    return num in set(primeTableNumber)

def GoldbachResult(n):
  ans = 0
  for i in range(2,int(n/2)):
    if (IsPrime(i) and IsPrime(n-i)):
      ans = ans+1
  return ans

def RandomInt(K):
  random_num = random.sample(range(K),10)
  return random_num

def RandomChoice(number,found,each_time):
  copy_list = list(set(primeTableNumber) - set(found))
  value_list = [n for n in copy_list if n<number]
  return random.sample(value_list, each_time)

def GA(number):

  found = []
  counter = 0
  total = int(number/25) #random  1/200
  epoch = 5
  each_time = int(total/5)

  for e in range(epoch):
    random_value = RandomChoice(number,found,each_time)
    print (f"random_value : {random_value} / \n  each_time : {each_time}")
    for t in range(each_time):
      # if (t%10 == 0): print(f"each_time : {t} / epoch : {e}")
      if (IsPrime(number - random_value[t])):
        # 計數+1
        counter = counter+1

        # 已找到的數另外剔除
        found.append(random_value[t])
        found.append(number - random_value[t])

        if (t%10 == 0):
          print(f"current : (epch){e}, (each_time){t}  /number of t : {random_value[t]} / current num : {number}")
  return counter

def G4(n) -> int:
  return int((n / np.log((n/2))**2)+0.5)

### new zone

def SplitDigit(num):
  digitList = [int(d) for d in str(num)]
  digitList = [0] * (7 - len(digitList)) + digitList #百萬
  return digitList


## heuristic random

###note:
evaluation 的時候需要有一個解，比如說
a + 2b + 3c + 4d = 30 這個問題，f(x) = ((a + 2b + 3c + 4d) - 30)<br>
同樣G4是不是也要有一個這樣的方法？<br>
G4(x) = ... = 25 ， 所以 G4(x) = ... - 25<br>
G4(10000) = n/(np.log(n/2))**2 = 138

## 0522<br>
GA

In [ ]:
import bisect

In [ ]:
target = 100000 # random choice in goldbach number
pair = []
primes = list(FindPrimeListLessThanNumber(target))
mutaionRate = 0.05

In [1]:
def SplitDigit(numList):
  digitList = []
  for i in numList:
    tmp = [int(d) for d in str(i)]
    tmp = ([0] * (7 - len(tmp))) + tmp #百萬tmp
    digitList.append(tmp)
  return digitList

def JoinDigit(num):
  """
  將數字列表合併成一個數字。

  Args:
  digits: 要合併的數字列表。

  Returns:
  合併後的數字。
  """
  print (f"num = {num}")
  digitList = []
  for item in num:
    print (f"item : {item}")
    tmp= int(''.join(map(str, item)))
    digitList.append(tmp)
  return digitList

def FindPrimeListLessThanNumber(n):
  """
  建立一個質數表，該質數表最大數字為n
  """
  index = bisect.bisect_left(primeTableNumber, n)
  return primeTableNumber[:index]

def IsPrime(num):
    """檢查一個數字是否為質數"""
    return num in set(primeTableNumber)

def InitChromosome(target):
  """
  從小於n的質數表中抽取10分之1個數
  """
  total = int(len(primes)/10 + 0.5)
  population = random.sample(primes, total)
  return (population)

def Evaluation(chromesome):
  """
  演化。演化的方法是確認是否為質數
  確認目前的 chromosome 所有的數字（item）與目標之差（target - item）是否為質數
  """
  elite = []
  for item in chromesome :
    tmp = target - item
    if (IsPrime(tmp) and IsPrime(item)):
      elite.append(item)
      elite.append(tmp)
  return elite

def CrossOver(inferior_chromesome, elite):
  """
  交配。
  將數字拆成一個個digits
  再把數字A[0000397] 跟 B[0015987] 奇數偶數位置交叉成 -> C[0005387]
  不斷交叉取代上一代的 chromosome
  """
  new_chromosome = []
  highest_digit = 7 - len(str(target))
  cross_over_times = len(elite) if len(elite) < len(inferior_chromesome) else len(inferior_chromesome)
  inferior_chromesome = SplitDigit(inferior_chromesome)
  elite = SplitDigit(elite)

  for i in range(cross_over_times):
    #A[0000397] B[0015987] -> C[0005387] 算到最高位，然後奇偶抄進child
    child = inferior_chromesome[i][:highest_digit]

    while(len(child) < 7):
      if (len(child) % 2 == 1):
        child = child + [elite[i][len(child)]]
      else:
        child = child + [inferior_chromesome[i][len(child)]]
    new_chromosome.append(child)

  # 如果數字不夠就 padding
  print (f"new_chromosome = {new_chromosome}")
  total = int(len(primes)/10 + 0.5)
  if (len(new_chromosome) < total):
    gen_chromosome = random.sample(primes, total - len(new_chromosome))
    new_chromosome = list(list(new_chromosome) + SplitDigit(gen_chromosome))

  print (f"gen_chromosome is {gen_chromosome}")
  new_chromosome = JoinDigit(new_chromosome)
  return new_chromosome

def Mutaion(chromesome):
  """
  如果隨機數小於 mutaionRate 則會將隨機一個位置的數字修改成 0-9 其中一個數字
  """
  for i in range(len(chromesome)) :
    if (random.random() < mutaionRate):
      tmp = [int(d) for d in str(chromesome[i])]
      tmp = ([0] * (7 - len(tmp))) + tmp #百萬tmp
      print (f"prepare to mutaion chromesome : {chromesome[i]} -> {tmp}")
      tmp[random.randint(0,6)] = random.randint(0,9)
      chromesome[i] = int(''.join(map(str, tmp)))
  return chromesome

def UinqueData(array):
  merged_list = list(chain.from_iterable(array))
  unique_items = list(set(merged_list))
  return unique_items

In [ ]:
"""
會迭代K次，K次之後
partition的預測數量是elite的不重複個數
"""
chromosome = InitChromosome(target)
print (f"init chromosome : {chromosome}")
K = 10 # genetic iteration

for i in range (K): # K is genetic iteration
  elite = Evaluation(chromosome) # elite is of one goldbach pair
  print (f"elite : {elite}")
  pair.append(elite)
  inferior_chromesome = list(set(chromosome) - set(elite))
  chromosome = CrossOver(inferior_chromesome, elite)
  chromosome = Mutaion(chromosome)
  print (f"iteration {i} choromsome is :{chromosome} and elite are {elite}")

pair = UinqueData(pair)
print (f"pair : {pair}")

In [ ]:
print (len(pair))

1278


In [ ]:
from itertools import chain

# 原始的雜亂陣列
array = [[47, 53], [], [53, 47, 29, 71], [47, 53, 53, 47, 89, 11], [], [3, 97, 47, 53], [], [47, 53], [11, 89], [71, 29], [41, 59], [47, 53], [11, 89, 17, 83], [47, 53, 89, 11], [11, 89, 47, 53], [3, 97, 29, 71], [29, 71, 59, 41], [], [17, 83], [], [17, 83, 3, 97], [59, 41], [89, 11, 29, 71], [53, 47, 59, 41], [], [53, 47, 97, 3], [97, 3, 47, 53, 17, 83], [83, 17, 89, 11], [41, 59, 59, 41], [53, 47, 47, 53], [47, 53], [97, 3], [41, 59], [57, 43, 41, 59], [71, 29], [29, 71], [], [97, 3], [93, 7], [95, 5, 97, 3], [17, 83], [81, 19], [11, 89, 41, 59], [17, 83, 83, 17, 97, 3], [59, 41], [83, 17, 97, 3], [89, 11, 71, 29, 59, 41], [], [71, 29], [77, 23, 27, 73], [89, 11, 83, 17], [47, 53, 71, 29], [53, 47], [57, 43, 59, 41], [59, 41, 53, 47], [71, 29, 89, 11], [71, 29, 41, 59], [77, 23, 53, 47], [53, 47, 89, 11], [3, 97], [], [29, 71], [71, 29], [], [89, 11, 29, 71]]
print(UinqueData(array))


[3, 5, 7, 11, 17, 19, 23, 27, 29, 41, 43, 47, 53, 57, 59, 71, 73, 77, 81, 83, 89, 93, 95, 97]


In [ ]:
A = [1,3,5,7,9,11,13]
B = [1,3,5]
print (list(set(A)-set(B)))

[9, 11, 13, 7]


In [ ]:
A = [1,3,5,7]
tmp= int(''.join(map(str, A)))
print (tmp)

1357


In [ ]:
A  = [1,3,5,7,9]

for i in range(len(A)) :
  A[i] = A[i]+1
print (A)

[2, 4, 6, 8, 10]
